In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import enum
import heapq
import tracemalloc
import timeit
import csv

In [2]:
goal_state = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 0]])

In [3]:
goal_dict = {}
for i in range(3):
    for j in range(3):
        goal_dict[goal_state[i, j]] = np.array([i, j])

In [4]:
class Actions(enum.Enum):
    UP = 0
    RIGHT = 1
    DOWN = 2
    LEFT = 3  
    def __str__(self):
        if self == Actions.UP:
            return 'U'
        elif self == Actions.RIGHT:
            return 'R'
        elif self == Actions.DOWN:
            return 'D'
        elif self == Actions.LEFT:
            return 'L'

In [5]:
def find_zero(puzzle):
    return np.argwhere(puzzle == 0)

In [6]:
class State:
    def __init__(self, puzzle, action=None, x=0, y=0, parent=None, g=None, f=None):
        if action == None:
            x, y = find_zero(puzzle)[0]
        self.puzzle = puzzle
        self.g = g
        self.f = f
        self.x = x
        self.y = y
        self.action = action
        self.parent = parent
    def DoAction(self, action, a, b):
        puzzle = self.puzzle.copy()
        puzzle[self.x, self.y], puzzle[self.x+a, self.y+b] = puzzle[self.x+a, self.y+b], puzzle[self.x, self.y]
        state = State(puzzle, action, self.x+a, self.y+b, self)
        return state
    def __lt__(self, other):
        return self.f < other.f

In [7]:
def NextState(state, action: Actions):
    if action == Actions.UP:
        if state.x > 0:
            return state.DoAction(action, -1, 0)
    elif action == Actions.RIGHT:
        if state.y < 2:
            return state.DoAction(action, 0, +1)
    elif action == Actions.DOWN:
        if state.x < 2:
            return state.DoAction(action, +1, 0)
    elif action == Actions.LEFT:
        if state.y > 0:
            return state.DoAction(action, 0, -1)
    return None

In [8]:
def Heuristic(puzzle):
    h = 0
    for i in range(3):
        for j in range(3):
            if goal_state[i,j] != puzzle[i,j] :
                h += sum(abs(goal_dict[puzzle[i,j]] - [i, j]))
    return h

In [9]:
def hash_function(puzzle):
    return hash(tuple(map(tuple, puzzle)))

In [10]:
def get_solution(state: State, debug=False):
    solutions = []
    if debug:
        print(state.puzzle)
    while state.action != None:
        solutions += [state.action]
        state = state.parent
        if debug:
            print(state.puzzle)
    solutions.reverse()
    return solutions

In [11]:
def A_star(start_state, debug=False):
    heap = [start_state]
    visited = set()
    while heap:
        state = heapq.heappop(heap)
        if np.array_equal(state.puzzle, goal_state):
            return get_solution(state, debug)
        for action in Actions:
            next_state = NextState(state, action)
            if next_state == None:
                continue
            hashed_state = hash_function(next_state.puzzle)
            if hashed_state not in visited:
                next_state.g = state.g + 1
                h = Heuristic(next_state.puzzle)
                next_state.f = next_state.g + h
                heapq.heappush(heap, next_state)
                visited.add(hashed_state)
    return None

In [12]:
start_puzzle = np.array([1,2,3,0,7,6,5,4,8]).reshape(3,3)
start_state = State(start_puzzle, g=0)

In [13]:
A_star(start_state, True)

[[1 2 3]
 [4 5 6]
 [7 8 0]]
[[1 2 3]
 [4 5 6]
 [7 0 8]]
[[1 2 3]
 [4 5 6]
 [0 7 8]]
[[1 2 3]
 [0 5 6]
 [4 7 8]]
[[1 2 3]
 [5 0 6]
 [4 7 8]]
[[1 2 3]
 [5 7 6]
 [4 0 8]]
[[1 2 3]
 [5 7 6]
 [0 4 8]]
[[1 2 3]
 [0 7 6]
 [5 4 8]]


[<Actions.DOWN: 2>,
 <Actions.RIGHT: 1>,
 <Actions.UP: 0>,
 <Actions.LEFT: 3>,
 <Actions.DOWN: 2>,
 <Actions.RIGHT: 1>,
 <Actions.RIGHT: 1>]

In [14]:
def DFS(state, visited, i, max_i, debug=False):
    if np.array_equal(state.puzzle, goal_state):
        return get_solution(state, debug)
    if i >= max_i:
        return None
    for action in Actions:
        next_state = NextState(state, action)
        if next_state == None:
            continue
        hashed_state = hash_function(next_state.puzzle)
        if hashed_state not in visited:
            visited.add(hashed_state)
            ans = DFS(next_state, visited, i+1, max_i, debug)
            if ans != None:
                return ans
    return None

In [15]:
def solve_DFS(start_puzzle, debug=False):
    visited = set()
    start_state = State(start_puzzle)
    return DFS(start_state, visited, 0, 50, debug)

In [16]:
solve_DFS(start_puzzle, True)

[[1 2 3]
 [4 5 6]
 [7 8 0]]
[[1 2 3]
 [4 5 6]
 [7 0 8]]
[[1 2 3]
 [4 0 6]
 [7 5 8]]
[[1 2 3]
 [0 4 6]
 [7 5 8]]
[[1 2 3]
 [7 4 6]
 [0 5 8]]
[[1 2 3]
 [7 4 6]
 [5 0 8]]
[[1 2 3]
 [7 0 6]
 [5 4 8]]
[[1 2 3]
 [7 6 0]
 [5 4 8]]
[[1 2 3]
 [7 6 8]
 [5 4 0]]
[[1 2 3]
 [7 6 8]
 [5 0 4]]
[[1 2 3]
 [7 0 8]
 [5 6 4]]
[[1 2 3]
 [0 7 8]
 [5 6 4]]
[[0 2 3]
 [1 7 8]
 [5 6 4]]
[[2 0 3]
 [1 7 8]
 [5 6 4]]
[[2 7 3]
 [1 0 8]
 [5 6 4]]
[[2 7 3]
 [1 6 8]
 [5 0 4]]
[[2 7 3]
 [1 6 8]
 [5 4 0]]
[[2 7 3]
 [1 6 0]
 [5 4 8]]
[[2 7 3]
 [1 0 6]
 [5 4 8]]
[[2 7 3]
 [1 4 6]
 [5 0 8]]
[[2 7 3]
 [1 4 6]
 [5 8 0]]
[[2 7 3]
 [1 4 0]
 [5 8 6]]
[[2 7 0]
 [1 4 3]
 [5 8 6]]
[[2 0 7]
 [1 4 3]
 [5 8 6]]
[[2 4 7]
 [1 0 3]
 [5 8 6]]
[[2 4 7]
 [1 8 3]
 [5 0 6]]
[[2 4 7]
 [1 8 3]
 [5 6 0]]
[[2 4 7]
 [1 8 0]
 [5 6 3]]
[[2 4 0]
 [1 8 7]
 [5 6 3]]
[[2 0 4]
 [1 8 7]
 [5 6 3]]
[[2 8 4]
 [1 0 7]
 [5 6 3]]
[[2 8 4]
 [1 6 7]
 [5 0 3]]
[[2 8 4]
 [1 6 7]
 [5 3 0]]
[[2 8 4]
 [1 6 0]
 [5 3 7]]
[[2 8 0]
 [1 6 4]
 [5 3 7]]
[[2 0 8]
 [1 6 4]
 [

[<Actions.UP: 0>,
 <Actions.RIGHT: 1>,
 <Actions.RIGHT: 1>,
 <Actions.DOWN: 2>,
 <Actions.DOWN: 2>,
 <Actions.LEFT: 3>,
 <Actions.UP: 0>,
 <Actions.UP: 0>,
 <Actions.RIGHT: 1>,
 <Actions.DOWN: 2>,
 <Actions.DOWN: 2>,
 <Actions.LEFT: 3>,
 <Actions.UP: 0>,
 <Actions.UP: 0>,
 <Actions.RIGHT: 1>,
 <Actions.DOWN: 2>,
 <Actions.DOWN: 2>,
 <Actions.LEFT: 3>,
 <Actions.UP: 0>,
 <Actions.UP: 0>,
 <Actions.RIGHT: 1>,
 <Actions.DOWN: 2>,
 <Actions.DOWN: 2>,
 <Actions.LEFT: 3>,
 <Actions.UP: 0>,
 <Actions.UP: 0>,
 <Actions.RIGHT: 1>,
 <Actions.DOWN: 2>,
 <Actions.DOWN: 2>,
 <Actions.LEFT: 3>,
 <Actions.UP: 0>,
 <Actions.RIGHT: 1>,
 <Actions.DOWN: 2>,
 <Actions.LEFT: 3>,
 <Actions.UP: 0>,
 <Actions.UP: 0>,
 <Actions.LEFT: 3>,
 <Actions.DOWN: 2>,
 <Actions.RIGHT: 1>,
 <Actions.DOWN: 2>,
 <Actions.RIGHT: 1>,
 <Actions.UP: 0>,
 <Actions.LEFT: 3>,
 <Actions.DOWN: 2>,
 <Actions.LEFT: 3>,
 <Actions.UP: 0>,
 <Actions.RIGHT: 1>,
 <Actions.DOWN: 2>,
 <Actions.RIGHT: 1>]

In [17]:
def BFS(state, debug=False):
    visited = set()
    queue = [state]
    while queue:
        state = queue.pop(0)
        for action in Actions:
            next_state = NextState(state, action)
            if next_state == None:
                continue
            if np.array_equal(state.puzzle, goal_state):
                return get_solution(state, debug)
            hashed_state = hash_function(next_state.puzzle)
            if hashed_state not in visited:
                visited.add(hashed_state)
                queue += [next_state]
    return None

In [18]:
start_state = State(start_puzzle)
BFS(start_state, True)

[[1 2 3]
 [4 5 6]
 [7 8 0]]
[[1 2 3]
 [4 5 6]
 [7 0 8]]
[[1 2 3]
 [4 0 6]
 [7 5 8]]
[[1 2 3]
 [0 4 6]
 [7 5 8]]
[[1 2 3]
 [7 4 6]
 [0 5 8]]
[[1 2 3]
 [7 4 6]
 [5 0 8]]
[[1 2 3]
 [7 0 6]
 [5 4 8]]
[[1 2 3]
 [0 7 6]
 [5 4 8]]


[<Actions.RIGHT: 1>,
 <Actions.DOWN: 2>,
 <Actions.LEFT: 3>,
 <Actions.UP: 0>,
 <Actions.RIGHT: 1>,
 <Actions.DOWN: 2>,
 <Actions.RIGHT: 1>]

In [19]:
def solve_IDS(start_puzzle, debug=False):
    start_state = State(start_puzzle)
    for i in range(2, 100):
        visited = set()
        ans = DFS(start_state, visited, 0, i, debug)
        if ans != None:
            return ans

In [20]:
solve_IDS(start_puzzle, True)

[[1 2 3]
 [4 5 6]
 [7 8 0]]
[[1 2 3]
 [4 5 6]
 [7 0 8]]
[[1 2 3]
 [4 0 6]
 [7 5 8]]
[[1 2 3]
 [0 4 6]
 [7 5 8]]
[[1 2 3]
 [7 4 6]
 [0 5 8]]
[[1 2 3]
 [7 4 6]
 [5 0 8]]
[[1 2 3]
 [7 0 6]
 [5 4 8]]
[[1 2 3]
 [0 7 6]
 [5 4 8]]
[[0 2 3]
 [1 7 6]
 [5 4 8]]
[[1 2 3]
 [0 7 6]
 [5 4 8]]


[<Actions.UP: 0>,
 <Actions.DOWN: 2>,
 <Actions.RIGHT: 1>,
 <Actions.DOWN: 2>,
 <Actions.LEFT: 3>,
 <Actions.UP: 0>,
 <Actions.RIGHT: 1>,
 <Actions.DOWN: 2>,
 <Actions.RIGHT: 1>]

In [21]:
def UCS(start_state, debug=False):
    heap = [start_state]
    visited = set()
    while heap:
        state = heapq.heappop(heap)
        if np.array_equal(state.puzzle, goal_state):
            return get_solution(state, debug)
        for action in Actions:
            next_state = NextState(state, action)
            if next_state == None:
                continue
            hashed_state = hash_function(next_state.puzzle)
            if hashed_state not in visited:
                next_state.f = state.f + 1
                heapq.heappush(heap, next_state)
                visited.add(hashed_state)
    return None

In [22]:
def solve_UCS(start_puzzle, debug=False):
    start_state = State(start_puzzle, f=0)
    return UCS(start_state, debug)

In [23]:
solve_UCS(start_puzzle, True)

[[1 2 3]
 [4 5 6]
 [7 8 0]]
[[1 2 3]
 [4 5 6]
 [7 0 8]]
[[1 2 3]
 [4 0 6]
 [7 5 8]]
[[1 2 3]
 [0 4 6]
 [7 5 8]]
[[1 2 3]
 [7 4 6]
 [0 5 8]]
[[1 2 3]
 [7 4 6]
 [5 0 8]]
[[1 2 3]
 [7 0 6]
 [5 4 8]]
[[1 2 3]
 [0 7 6]
 [5 4 8]]


[<Actions.RIGHT: 1>,
 <Actions.DOWN: 2>,
 <Actions.LEFT: 3>,
 <Actions.UP: 0>,
 <Actions.RIGHT: 1>,
 <Actions.DOWN: 2>,
 <Actions.RIGHT: 1>]

In [24]:
class Methods(enum.Enum):
    A_star = 1
    DFS = 2
    BFS = 3
    IDS = 4
    UCS = 5

In [25]:
def str_solution(solution):
    for i in range(len(solution)):
        solution[i] = str(solution[i])
    return "".join(solution)

In [26]:
def solve(start_puzzle, method:Methods, debug=False):
    tracemalloc.start()
    start_time = timeit.default_timer()
    if method == Methods.A_star: 
        start_state = State(start_puzzle, g=0)
        solution = A_star(start_state, debug)
    elif method == Methods.DFS:
        solution = solve_DFS(start_puzzle, debug)
    elif method == Methods.BFS:
        start_state = State(start_puzzle)
        solution = BFS(start_state, debug)
    elif method == Methods.IDS:
        solution = solve_IDS(start_puzzle, debug)
    elif method == Methods.UCS:
        solution = solve_UCS(start_puzzle, debug)
    current, peak = tracemalloc.get_traced_memory()
    diff_time = timeit.default_timer() - start_time
    tracemalloc.stop()
    if solution == None:
        solution = ["N/A"]
    return str_solution(solution), diff_time, peak

In [27]:
solve(start_puzzle, Methods.DFS, True)

[[1 2 3]
 [4 5 6]
 [7 8 0]]
[[1 2 3]
 [4 5 6]
 [7 0 8]]
[[1 2 3]
 [4 0 6]
 [7 5 8]]
[[1 2 3]
 [0 4 6]
 [7 5 8]]
[[1 2 3]
 [7 4 6]
 [0 5 8]]
[[1 2 3]
 [7 4 6]
 [5 0 8]]
[[1 2 3]
 [7 0 6]
 [5 4 8]]
[[1 2 3]
 [7 6 0]
 [5 4 8]]
[[1 2 3]
 [7 6 8]
 [5 4 0]]
[[1 2 3]
 [7 6 8]
 [5 0 4]]
[[1 2 3]
 [7 0 8]
 [5 6 4]]
[[1 2 3]
 [0 7 8]
 [5 6 4]]
[[0 2 3]
 [1 7 8]
 [5 6 4]]
[[2 0 3]
 [1 7 8]
 [5 6 4]]
[[2 7 3]
 [1 0 8]
 [5 6 4]]
[[2 7 3]
 [1 6 8]
 [5 0 4]]
[[2 7 3]
 [1 6 8]
 [5 4 0]]
[[2 7 3]
 [1 6 0]
 [5 4 8]]
[[2 7 3]
 [1 0 6]
 [5 4 8]]
[[2 7 3]
 [1 4 6]
 [5 0 8]]
[[2 7 3]
 [1 4 6]
 [5 8 0]]
[[2 7 3]
 [1 4 0]
 [5 8 6]]
[[2 7 0]
 [1 4 3]
 [5 8 6]]
[[2 0 7]
 [1 4 3]
 [5 8 6]]
[[2 4 7]
 [1 0 3]
 [5 8 6]]
[[2 4 7]
 [1 8 3]
 [5 0 6]]
[[2 4 7]
 [1 8 3]
 [5 6 0]]
[[2 4 7]
 [1 8 0]
 [5 6 3]]
[[2 4 0]
 [1 8 7]
 [5 6 3]]
[[2 0 4]
 [1 8 7]
 [5 6 3]]
[[2 8 4]
 [1 0 7]
 [5 6 3]]
[[2 8 4]
 [1 6 7]
 [5 0 3]]
[[2 8 4]
 [1 6 7]
 [5 3 0]]
[[2 8 4]
 [1 6 0]
 [5 3 7]]
[[2 8 0]
 [1 6 4]
 [5 3 7]]
[[2 0 8]
 [1 6 4]
 [

('URRDDLUURDDLUURDDLUURDDLUURDDLURDLUULDRDRULDLURDR',
 0.302599900009227,
 330080)

In [28]:
examples = np.array([[ 1,2,3,0,7,6,5,4,8 ],
[ 0,4,1,2,5,3,7,8,6 ],
[ 4,1,3,0,2,6,7,5,8 ],
[ 1,2,3,0,4,8,7,6,5 ],
[ 1,2,0,4,8,3,7,6,5 ],
[ 1,0,2,4,6,3,7,5,8 ],
[ 0,1,2,4,5,3,7,8,6 ],
[ 1,2,3,0,4,5,7,8,6 ],
[ 1,2,3,4,0,5,7,8,6 ],
[ 1,2,3,4,5,0,7,8,6 ],
[ 0,1,3,4,2,5,7,8,6 ],
[ 2,3,5,1,0,4,7,8,6 ],
[ 1,6,2,5,3,0,4,7,8 ],
[ 1,8,2,0,4,3,7,6,5 ],
[ 2,5,3,4,1,6,0,7,8 ],
[ 1,2,3,4,6,8,7,5,0 ],
[ 1,6,2,5,7,3,0,4,8 ],
[ 0,4,1,5,3,2,7,8,6 ],
[ 0,5,2,1,8,3,4,7,6 ],
[ 1,2,3,0,4,6,7,5,8 ],
[ 1,3,5,7,2,6,8,0,4 ],
[ 4,1,2,3,0,6,5,7,8 ],
[ 4,3,1,0,7,2,8,5,6 ],
[ 5,2,1,4,8,3,7,6,0 ],
[ 2,0,8,1,3,5,4,6,7 ],
[ 3,5,6,1,4,8,0,7,2 ],
[ 1,0,2,7,5,4,8,6,3 ],
[ 5,1,8,2,7,3,4,0,6 ],
[ 4,3,0,6,1,8,2,7,5 ],
[ 2,4,3,1,6,5,8,0,7 ],
[ 1,2,3,6,4,5,7,8,0 ],
[ 3,1,2,4,5,6,7,8,0 ],
[ 1,2,3,4,8,7,6,5,0 ],
[ 1,3,2,5,4,6,7,8,0 ],
[ 1,4,2,6,5,8,7,3,0 ],
[ 2,1,3,4,5,6,8,7,0 ],
[ 2,3,1,6,5,4,8,7,0 ],
[ 2,3,1,6,4,5,7,8,0 ],
[ 1,2,3,6,5,4,8,7,0 ],
[ 1,2,3,6,5,4,0,8,7 ],
[ 4,5,3,2,8,0,6,7,1 ],
[ 4,5,3,2,1,0,8,7,6 ],
[ 1,2,4,3,5,0,8,7,6 ],
[ 1,2,4,3,5,8,7,0,6 ],
[ 2,1,3,4,5,8,7,0,6 ],
[ 1,3,5,8,7,0,6,2,4 ],
[ 4,3,1,6,5,8,0,2,7 ],
[ 7,0,4,8,5,1,6,3,2 ],
[ 8,7,2,1,5,0,4,6,3 ],
[ 8,3,5,6,4,2,1,0,7 ],
[ 1,6,4,0,3,5,8,2,7 ],
[ 6,3,8,5,4,1,7,2,0 ],
[ 5,8,7,1,4,6,3,0,2 ],
[ 2,8,5,3,6,1,7,0,4 ],
[ 8,7,6,5,4,3,2,1,0 ]])

In [29]:
examples = examples.reshape(len(examples),3,3)
examples

array([[[1, 2, 3],
        [0, 7, 6],
        [5, 4, 8]],

       [[0, 4, 1],
        [2, 5, 3],
        [7, 8, 6]],

       [[4, 1, 3],
        [0, 2, 6],
        [7, 5, 8]],

       [[1, 2, 3],
        [0, 4, 8],
        [7, 6, 5]],

       [[1, 2, 0],
        [4, 8, 3],
        [7, 6, 5]],

       [[1, 0, 2],
        [4, 6, 3],
        [7, 5, 8]],

       [[0, 1, 2],
        [4, 5, 3],
        [7, 8, 6]],

       [[1, 2, 3],
        [0, 4, 5],
        [7, 8, 6]],

       [[1, 2, 3],
        [4, 0, 5],
        [7, 8, 6]],

       [[1, 2, 3],
        [4, 5, 0],
        [7, 8, 6]],

       [[0, 1, 3],
        [4, 2, 5],
        [7, 8, 6]],

       [[2, 3, 5],
        [1, 0, 4],
        [7, 8, 6]],

       [[1, 6, 2],
        [5, 3, 0],
        [4, 7, 8]],

       [[1, 8, 2],
        [0, 4, 3],
        [7, 6, 5]],

       [[2, 5, 3],
        [4, 1, 6],
        [0, 7, 8]],

       [[1, 2, 3],
        [4, 6, 8],
        [7, 5, 0]],

       [[1, 6, 2],
        [5, 7, 3],
        [0, 4, 8]]

In [35]:
with open('result.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["Example", "A_Star-Answer", "A_Star-Time", "A_Star-MemPeak", "DFS-Answer", "DFS-Time", "DFS-MemPeak", "BFS-Answer",
                    "BFS-Time", "BFS-MemPeak", "IDS-Answer", "IDS-Time", "IDS-MemPeak", "UCS-Answer", "UCS-Time", "UCS-MemPeak"])
    for i, example in enumerate(examples):
        print("Example: ", i)
        a_star = solve(example, Methods.A_star)
        dfs = solve(example, Methods.DFS)
        bfs = solve(example, Methods.BFS)
        ids = solve(example, Methods.IDS)
        ucs = solve(example, Methods.UCS)
        writer.writerow([example, *a_star, *dfs, *bfs, *ids, *ucs])

Example:  0
Example:  1
Example:  2
Example:  3
Example:  4
Example:  5
Example:  6
Example:  7
Example:  8
Example:  9
Example:  10
Example:  11
Example:  12
Example:  13
Example:  14
Example:  15
Example:  16
Example:  17
Example:  18
Example:  19
Example:  20
Example:  21
Example:  22
Example:  23
Example:  24
Example:  25
Example:  26
Example:  27
Example:  28
Example:  29
Example:  30
Example:  31
Example:  32
Example:  33
Example:  34
Example:  35
Example:  36
Example:  37
Example:  38
Example:  39
Example:  40
Example:  41
Example:  42
Example:  43
Example:  44
Example:  45
Example:  46
Example:  47
Example:  48
Example:  49
Example:  50
Example:  51
Example:  52
Example:  53
Example:  54
